In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

from utils import get_subsample_method_name, get_results_path, get_retriever_output_path, load_results

/home/rjha/miniforge3/envs/dataset_subsample/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
dataset = "nfcorpus"
neg_retriever = "BM25s"
subsample_top_k = 250
subsample_method = get_subsample_method_name(neg_retriever, subsample_top_k)
retriever = "BM25s"
full = load_results(get_results_path(dataset=dataset, subsample_method=subsample_method, retriever=retriever))
full["dataset"] = dataset
full["subsample_method"] = subsample_method
full["neg_retriever"] = neg_retriever
full["subsample_top_k"] = subsample_top_k
full["retriever"] = retriever
cols = full.columns.to_list()
full = full[cols[-5:] + cols[1:-5]]


In [15]:
full

,dataset,subsample_method,neg_retriever,subsample_top_k,retriever,nDCG@10,nDCG@100,nDCG@1000,R@1,R@10,R@100,R@1000,RR@10,RR@100,RR@1000,Success@1,Success@5,Success@10,Success@50,Success@100
0,nfcorpus,BM25s_top_250,BM25s,250,BM25s,0.287909,0.244958,0.315356,0.035613,0.127115,0.231426,0.450916,0.486411,0.491373,0.491677,0.407407,0.592593,0.657407,0.740741,0.759259


In [18]:
"BM25s_top_250".split("_", maxsplit=2)

['BM25s', 'top', '250']

In [20]:
full["num_docs"] = 42

In [9]:
retrievers = [
    "snowflake-arctic-embed-m-long",
    "gte-base-en-v1.5",
    "all-mpnet-base-v2",
    "bge-base-en-v1.5",
    "e5-base-v2",
    "jina-embeddings-v2-base-en-flash",
    "BM25s",
]
datasets = ["nfcorpus"]
top_ks = [1, 10, 50, 100, 250, 500, 750, 1000]

full = pd.concat([
    load_results(get_results_path(
        dataset=dataset,
        subsample_method="full",
        retriever=retriever,
    ))
    for retriever in retrievers
    for dataset in datasets
] +[
    load_results(get_results_path(
        dataset=dataset,
        subsample_method=get_subsample_method_name(neg_retriever, top_k),
        retriever=retriever,
    ))
    for retriever in retrievers
    for neg_retriever in retrievers
    for dataset in datasets
    for top_k in top_ks
])


In [10]:
full.to_csv("foobar.csv")

In [7]:
filtered_full = full[(full.dataset == "nfcorpus") & (full.retriever == "BM25s")]

AttributeError: 'DataFrame' object has no attribute 'dataset'

In [67]:
filtered_full

,dataset,retriever,neg_retriever_name,subsample_top_k,num_docs,nDCG@10,nDCG@100,nDCG@1000,R@1,R@10,R@100,R@1000,RR@10,RR@100,RR@1000,Success@1,Success@5,Success@10,Success@50,Success@100
0,nfcorpus,BM25s,full,0,3633,0.287924,0.244999,0.315395,0.035613,0.127115,0.23147,0.451041,0.486411,0.491373,0.491677,0.407407,0.592593,0.657407,0.740741,0.759259
